In [ ]:
from src.dataloader import DataLoader
from utils.utils import flatten_list, write_list_to_file, read_list_from_file, preprocess_text
from bs4 import BeautifulSoup
from src.faiss.flat_idx import FlatIdx
from src.encoder.tf_idf import Encoder
from src.post_processing import idk
import pandas as pd
from src.eval import Eval
import numpy as np
from fuzzywuzzy import fuzz
import json

## read all the table data

In [ ]:
def html_parser(html_content):
    soup = BeautifulSoup(html_content, "html.parser")
    text = soup.get_text(separator=" ", strip=True)
    return(text)

In [ ]:
directory_path = r'C:\Users\J C SINGLA\Downloads\External - take_home_challenge_(withJSONs)\take_home_challenge_(withJSONs)\raw_text'
loader = DataLoader(directory_path, "")

In [ ]:
all_data, filenames = loader.read_json_from_folders()

In [ ]:
all_tables = [(k,v) for (k,v) in zip(all_data, filenames) if "<table>" in k]

## merge with other data

In [ ]:
save_path = r"C:\Users\J C SINGLA\Downloads\External - take_home_challenge_(withJSONs)\take_home_challenge_(withJSONs)\data"

In [ ]:
all_data_sherpa = read_list_from_file(save_path, "sherpa_paras")
filenames_sherpa = read_list_from_file(save_path, "sherpa_paras_filenames")
assert (len(all_data_sherpa)==len(filenames_sherpa))

In [ ]:
filenames_tables = [i[1] for i in all_tables]
all_tables = [html_parser(i[0]) for i in all_tables]
assert(len(filenames_tables)==len(all_tables))

In [ ]:
all_data_sherpa = all_tables + all_data_sherpa
filenames_sherpa = filenames_tables + filenames_sherpa

In [ ]:
assert (len(all_data_sherpa)==len(filenames_sherpa))
print (len(all_data_sherpa))

In [ ]:
ground_truth_path = r"C:\Users\J C SINGLA\Downloads\External - take_home_challenge_(withJSONs)\take_home_challenge_(withJSONs)\document_questions.xlsx"
ground_truth = pd.read_excel(ground_truth_path)
ground_truth_text = ground_truth[ground_truth["complexity"].isin(["table", "text","no raw_text"])].copy()  
test_data = list(ground_truth_text["relevant questions"])
test_labels = list(ground_truth_text["answer"])

In [ ]:
documents = list(ground_truth["document"])

In [ ]:
complexity = list(ground_truth["complexity"])